In [34]:

import pandas as pd
import requests

In [35]:

# link for extract html data
def getdata(url):
    r = requests.get(url)
    return r.text

In [39]:

def fetch_player_stats_single_year(year: int):
    df = pd.read_html(getdata(f'https://www.basketball-reference.com/leagues/NBA_{year}_per_game.html'))[0]
    df_adv = pd.read_html(getdata(f'https://www.basketball-reference.com/leagues/NBA_{year}_advanced.html'))[0]

    df = df.drop(df[df['Player'] == 'Player'].index)

    df_adv = df_adv.drop(df_adv[df_adv['Player'] == 'Player'].index)

    new = pd.merge(df, df_adv, on=['Player', 'Age', 'Tm', 'Pos'])

    player_stats_list = ['Player', 'Pos', 'Age', 'Tm', 'G_x', 'GS', 'MP_x', 'FG', 'FGA',
                         'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
                         'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
                         'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%',
                         'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS',
                         'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']

    player_stats_list_correct = ['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
                                 '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
                                 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PER',
                                 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%',
                                 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM',
                                 'VORP']

    # cuts down the columns to what is listed in player_stats_list_correct
    new = new[player_stats_list]
    new.columns = player_stats_list_correct

    # This gets rid of traded players entirely
    # new = new.drop_duplicates(subset='Player', keep=False).reset_index(drop=True)
    new.drop_duplicates(subset=['Player'], keep='first', inplace=True)
    # Fills nan's with 0
    new.fillna(value=0, inplace=True)

    # Adds a year column
    new['Year'] = year

    return new

    # if not os.path.isdir(f'Data/Test Sets/{datetime.now().strftime("%Y")}'):
    #     os.mkdir(f'Data/Test Sets/{datetime.now().strftime("%Y")}')
    #
    # new.to_csv(f'Data/Test Sets/{datetime.now().strftime("%Y")}/{datetime.now().strftime("%Y%m%d")}_player_stats.csv')
    # print(datetime.now().strftime("%m/%d/%Y"), 'Complete')




In [37]:
player_list = []
for i in range(1980, 2023):
    print(f'Done - {i}')
    player_list.append(fetch_player_stats_single_year(i))


df = pd.concat(player_list)

In [38]:
df.shape

In [12]:


# df['All_Star'] = df.Player.str.contains('\*')
#
df.Player = df.Player.str.strip('\*')
# df['All_Star']


In [32]:
import numpy as np

# df.All_Star = np.where(df.All_Star == True,False,True)


df[df.Player == 'Stephen Curry']

In [15]:
df2 = df.copy()

In [16]:
df2[(df2.Year == 1987) & (df2.Player == 'Michael Jordan')]

In [17]:

# For some reason the 1997 MVP file doesn't contain Karl (winner) or Michael (2nd) by default. They were added to the file manually.

awards_df = list()
for year in range(1980, 2023):
    awards_df.append(
        pd.read_csv(f'/Users/chaseallbright/Desktop/content/Cleaned Awards/MVP/{year}-Most Valuable Player.csv',
                    index_col=0))

awards_df = pd.concat(awards_df)

In [18]:
awards_df[(awards_df.MVP_Player == 'Karl Malone') & (awards_df.Year == 1997)]

In [19]:
df = df2.copy()

df.loc[:, ['Player', 'Pos', 'Tm']] = df[['Player', 'Pos', 'Tm']].astype('string')
df.loc[:, ['Age', 'G', 'GS', 'Year']] = df[['Age', 'G', 'GS', 'Year']].astype('int')
df.loc[:, df.drop(columns=(['Age', 'G', 'GS', 'Year', 'Player', 'Pos', 'Tm'])).columns] = df.loc[:, df.drop(
    columns=(['Age', 'G', 'GS', 'Year', 'Player', 'Pos', 'Tm'])).columns].astype('float64')

In [20]:
full = pd.merge(df, awards_df, right_on=['MVP_Player', 'MVP_Age'], left_on=['Player', 'Age'], how='left')
full = full.drop(columns=['Year_y', 'MVP_Player', 'MVP_Age'], )

In [21]:
full = full.fillna(0)

In [22]:
full.isna().sum()

In [23]:
full.Pos.unique()

In [24]:
pos_map = {'PG': 1,
           'SG': 2,
           'SF': 3,
           'PF': 4,
           'C': 5,
           'SG-SF': 2,
           'SG-PG': 2,
           'PG-SG': 1,
           'SF-PF': 3,
           'PG-SF': 1,
           'SG-PF': 2,
           'SF-C': 3,
           'SF-SG': 3,
           'PF-SF': 4,
           'C-PF': 5,
           'PF-C': 4,
           'SG-PG-SF': 2,
           }

full.Pos = full.Pos.map(pos_map).astype('int')

In [25]:
full.Pos.unique()

In [26]:
full['Year_x'].unique()


In [33]:
pd.read_csv('Full Dataset.csv',index_col=0).drop(columns='All_Star',inplace=True)

full.to_csv('Full Dataset.csv')

In [22]:
full[(full.Year_x == 1997) & (full.MVP_Share != 0)]